In [ ]:
!python --version

### PIP

In [ ]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [ ]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

### AWS CLI and AWS Python SDK (boto3)

In [ ]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

### Install SageMaker

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

### Install PyAthena

In [ ]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

### Install Matplotlib and Seaborn

In [ ]:
!pip install --disable-pip-version-check -q matplotlib==3.1.3

In [ ]:
!pip install --disable-pip-version-check -q seaborn==0.10.0

### Install Imblearn

In [ ]:
pip install imblearn

### Install PrettyTable

In [ ]:
pip install prettytable

### Check Environment was created correctly  
_Note:  This workshop requires SageMaker Studio and will not work properly in classic SageMaker Notebooks._

In [ ]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name="ec2", region_name=region)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
import json

notebook_instance_name = None

try:
    with open("/opt/ml/metadata/resource-metadata.json") as notebook_info:
        data = json.load(notebook_info)
        domain_id = data["DomainId"]
        resource_arn = data["ResourceArn"]
        region = resource_arn.split(":")[3]
        name = data["ResourceName"]
    print("DomainId: {}".format(domain_id))
    print("Name: {}".format(name))
except:
    print("+++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: COULD NOT RETRIEVE THE METADATA.")
    print("+++++++++++++++++++++++++++++++++++++++++")

In [ ]:
describe_domain_response = sm.describe_domain(DomainId=domain_id)
print(describe_domain_response["Status"])

In [ ]:
try:
    get_status_response = sm.get_sagemaker_servicecatalog_portfolio_status()
    print(get_status_response["Status"])
except:
    pass

### Summary: Check All Required Settings Are Set Correctly

In [ ]:
if (
    describe_domain_response["Status"] == "InService"
    and get_status_response["Status"] == "Enabled"
    and "datascience" in name
):
    setup_instance_check_passed = True
    print("[OK] Checks passed!  Great Job!!  Please Continue.")
else:
    setup_instance_check_passed = False
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: WE HAVE IDENTIFIED A MISCONFIGURATION.")
    print(describe_domain_response["Status"])
    print(get_status_response["Status"])
    print(name)
    print("+++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(setup_instance_check_passed)

In [ ]:
%store setup_instance_check_passed

In [ ]:
%store

### Create S3 Bucket

In [ ]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:
setup_s3_bucket_passed = False

In [ ]:
print("Default bucket: {}".format(bucket))

### Verify S3_BUCKET Bucket Creation

In [ ]:
%%bash

aws s3 ls s3://${bucket}/

In [ ]:
%store setup_s3_bucket_passed

In [ ]:
%store

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

### Update IAM Roles and Policies

In [ ]:
import boto3
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

### Get SageMaker Execution Role Name

In [ ]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

In [ ]:
setup_iam_roles_passed = False

### **Pre-Requisite:  SageMaker notebook instance ExecutionRole contains `IAMFullAccess` Policy.**

In [ ]:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        break

setup_iam_roles_passed = True
print("[OK] You are all set up to continue with this workshop!")

In [ ]:
if not admin:
    pre_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

    required_policies = ["IAMFullAccess"]

    for pre_policy in pre_policies:
        for role_req in required_policies:
            if pre_policy["PolicyName"] == role_req:
                print("Attached: {}".format(pre_policy["PolicyName"]))
                try:
                    required_policies.remove(pre_policy["PolicyName"])
                except:
                    pass

    if len(required_policies) > 0:
        print(
            "*************** [ERROR] You need to attach the following policies in order to continue with this workshop *****************\n"
        )
        for required_policy in required_policies:
            print("Not Attached: {}".format(required_policy))
    else:
        print("[OK] You are all set to continue with this notebook!")
else:
    print("[OK] You are all set to continue with this notebook!")

**If you see an ERROR message ^^ above ^^, please attach the IAMFullAccess Policy to the SageMaker notebook instance ExecutionRole.**

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AdministratorAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonSageMakerFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "IAMFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonS3FullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "ComprehendFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonAthenaFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "SecretsManagerReadWrite"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonRedshiftFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonEC2ContainerRegistryFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AWSStepFunctionsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisFirehoseFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

In [ ]:
from botocore.exceptions import ClientError

try:
    policy = "AmazonKinesisAnalyticsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)

### Final Check

In [ ]:
# role = iam.get_role(RoleName=role_name)
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

required_policies = [
    "AdministratorAccess",
    "SecretsManagerReadWrite",
    "IAMFullAccess",
    "AmazonS3FullAccess",
    "AmazonAthenaFullAccess",
    "ComprehendFullAccess",
    "AmazonEC2ContainerRegistryFullAccess",
    "AmazonRedshiftFullAccess",
    "AWSStepFunctionsFullAccess",
    "AmazonSageMakerFullAccess",
    "AmazonKinesisFullAccess",
    "AmazonKinesisFirehoseFullAccess",
    "AmazonKinesisAnalyticsFullAccess",
]

admin = False

for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            break
    else:
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            pass

if not admin and len(required_policies) > 0:
    print("*************** [ERROR] RE-RUN THIS NOTEBOOK *****************")
    for required_policy in required_policies:
        print("Not Attached: {}".format(required_policy))
else:
    setup_iam_roles_passed = True
    print("[OK] You are all set up to continue with this workshop!")

In [ ]:
%store setup_iam_roles_passed

In [ ]:
%store

### Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}